In [8]:
import torch
from typing import List
import os
import sys

from os.path import join

sys.path.insert(0, os.getcwd())

from stable_diffusion2.model.clip_text_embedder import CLIPTextEmbedder
from stable_diffusion2.utils.utils import check_device, get_memory_status

EMBEDDED_PROMPTS_DIR = os.path.abspath("./input/embedded_prompts/")
NULL_PROMPT = ""
PROMPTS = "An old photo of a computer scientist"
NUM_IMAGES = 10
SEED = 2982
NOISE_MULTIPLIER = 0.1

os.makedirs(EMBEDDED_PROMPTS_DIR, exist_ok=True)
DEVICE = input("Set device: 'cuda:i' or 'cpu'")
DEVICE = check_device(DEVICE)

INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.


In [2]:
def embed_and_save_prompts(prompts: list, null_prompt=NULL_PROMPT):
    null_prompt = null_prompt
    prompts = prompts

    clip_text_embedder = CLIPTextEmbedder(device=check_device())
    clip_text_embedder.load_submodels()

    null_cond = clip_text_embedder(null_prompt)
    torch.save(null_cond, join(EMBEDDED_PROMPTS_DIR, "null_cond.pt"))
    print(
        "Null prompt embedding saved at: ",
        f"{join(EMBEDDED_PROMPTS_DIR, 'null_cond.pt')}",
    )

    embedded_prompts = clip_text_embedder(prompts)
    torch.save(embedded_prompts, join(EMBEDDED_PROMPTS_DIR, "embedded_prompts.pt"))

    print(
        "Prompts embeddings saved at: ",
        f"{join(EMBEDDED_PROMPTS_DIR, 'embedded_prompts.pt')}",
    )

    get_memory_status()
    clip_text_embedder.to("cpu")
    del clip_text_embedder
    torch.cuda.empty_cache()
    get_memory_status()

    return embedded_prompts, null_cond

In [3]:
embedded_prompts, null_cond = embed_and_save_prompts(PROMPTS)

INFO: `device` is None. Falling back to current device.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
INFO: Using CUDA device 0: NVIDIA GeForce RTX 3080 Ti.
Null prompt embedding saved at:  c:\Users\igor-\.cloned\kcg-ml-sd1p4\input\embedded_prompts\null_cond.pt
Prompts embeddings saved at:  c:\Users\igor-\.cloned\kcg-ml-sd1p4\input\embedded_prompts\embedded_prompts.pt
Total: 12287 MiB
Free: 10376 MiB
Used: 1911 MiB
Total: 12287 MiB
Free: 10522 MiB
Used: 1765 MiB


In [21]:
# check the shape of the embedded prompts
embedding_shape = tuple(embedded_prompts.shape)
embedding_shape

(1, 77, 768)

In [22]:
# check mean and std to use the same for the noise generation
# one idea is to use one distribution per position (in the 77 positions)
# in this case we would check the mean and std along dimension 2
# embedded_prompts.mean(dim=2), embedded_prompts.std(dim=2)
embedding_mean, embedding_std = embedded_prompts.mean(), embedded_prompts.std()
embedding_mean, embedding_std

(tensor(-0.1092, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(1.0230, device='cuda:0', grad_fn=<StdBackward0>))

In [24]:
# generate noise and add to the embedded prompt
generator = torch.Generator(device=DEVICE).manual_seed(SEED)
noise = torch.normal(
    mean=embedding_mean.item(),
    std=embedding_std.item(),
    size=embedding_shape,
    device=DEVICE,
    generator=generator,
)
noise.shape

torch.Size([1, 77, 768])

In [25]:
embedding_e = embedded_prompts + noise